In [1]:
# 到了这篇就要注意豆瓣的防爬机制了

In [2]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd

import pprint

In [3]:
#构造分页数字列表
page_indexs=range(0,250,25)

In [4]:
headers={'User-Agent':'Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Trident/5.0'
         ,'Sec - Fetch - Site': 'same - origin'

         }

In [5]:
def download_all_htmls():
    '''
    下载所有的列表页面的HTMl,用于后续的分析
    '''
    htmls=[]
    for idx in page_indexs:
        url=f"https://movie.douban.com/top250?start={idx}&filter="
        print('craw html:',url)
        r=requests.get(url,headers=headers)
        if r.status_code !=200:
            raise Exception("Error")
        htmls.append(r.text)#它返回一个字符串，包含标签内的所有文本内容  
    return htmls

In [6]:
#执行爬取
htmls=download_all_htmls()

craw html: https://movie.douban.com/top250?start=0&filter=
craw html: https://movie.douban.com/top250?start=25&filter=
craw html: https://movie.douban.com/top250?start=50&filter=
craw html: https://movie.douban.com/top250?start=75&filter=
craw html: https://movie.douban.com/top250?start=100&filter=
craw html: https://movie.douban.com/top250?start=125&filter=
craw html: https://movie.douban.com/top250?start=150&filter=
craw html: https://movie.douban.com/top250?start=175&filter=
craw html: https://movie.douban.com/top250?start=200&filter=
craw html: https://movie.douban.com/top250?start=225&filter=


In [7]:
htmls[0]

'<!DOCTYPE html>\n<html lang="zh-CN" class="ua-windows ua-ie9">\n<head>\n    <meta http-equiv="Content-Type" content="text/html; charset=utf-8">\n    <meta name="renderer" content="webkit">\n    <meta name="referrer" content="always">\n    <meta name="google-site-verification" content="ok0wCgT20tBBgo9_zat2iAcimtN4Ftf5ccsh092Xeyw" />\n    <title>\n豆瓣电影 Top 250\n</title>\n    \n    <meta name="baidu-site-verification" content="cZdR4xxR7RxmM4zE" />\n    <meta http-equiv="Pragma" content="no-cache">\n    <meta http-equiv="Expires" content="Sun, 6 Mar 2006 01:00:00 GMT">\n    \n    <link rel="apple-touch-icon" href="https://img1.doubanio.com/f/movie/d59b2715fdea4968a450ee5f6c95c7d7a2030065/pics/movie/apple-touch-icon.png">\n    <link href="https://img1.doubanio.com/f/vendors/02814fbb5bee25484516bd0a642af695f7ec5a83/css/douban.css" rel="stylesheet" type="text/css">\n    <link href="https://img1.doubanio.com/f/vendors/ee6598d46af0bc554cecec9bcbf525b9b0582cb0/css/separation/_all.css" rel="styl

In [8]:
def parse_singe_html(html):
    '''
    解析单个HTML,得到数据
    '''
    soup=BeautifulSoup(html,'html.parser')
    article_items=(
    soup.find('div',class_='article')
        .find('ol',class_='grid_view')
        .find_all('div',class_='item')
    )
    datas=[]
    for article_item in article_items:
        rank=article_item.find('div',class_='pic').find('em').get_text()
        info=article_item.find('div',class_='info')
        title=info.find('div',class_='hd').find('span',class_='title').get_text()
        stars=(
        info.find('div',class_='bd')
            .find('div',class_='star')
            .find_all('span')
        )
        rating_star=stars[0]['class'][0]
        rating_num=stars[1].get_text()
        comments=stars[3].get_text()
        
        datas.append(
        {
            'rank':rank,#排名
            'title':title,#电影名
            "rating_star":rating_star.replace('raring','').replace('-t',''),#五星评级
            'rating_num':rating_num,#评价分数
            'comments':comments.replace('人评价','')#评价人数
            
        }
        )
    return datas

In [9]:
pprint.pprint(parse_singe_html(htmls[0]))

[{'comments': '2919828',
  'rank': '1',
  'rating_num': '9.7',
  'rating_star': 'rating5',
  'title': '肖申克的救赎'},
 {'comments': '2155515',
  'rank': '2',
  'rating_num': '9.6',
  'rating_star': 'rating5',
  'title': '霸王别姬'},
 {'comments': '2176753',
  'rank': '3',
  'rating_num': '9.5',
  'rating_star': 'rating5',
  'title': '阿甘正传'},
 {'comments': '2209162',
  'rank': '4',
  'rating_num': '9.5',
  'rating_star': 'rating5',
  'title': '泰坦尼克号'},
 {'comments': '2310124',
  'rank': '5',
  'rating_num': '9.4',
  'rating_star': 'rating45',
  'title': '这个杀手不太冷'},
 {'comments': '2260626',
  'rank': '6',
  'rating_num': '9.4',
  'rating_star': 'rating45',
  'title': '千与千寻'},
 {'comments': '1335427',
  'rank': '7',
  'rating_num': '9.6',
  'rating_star': 'rating5',
  'title': '美丽人生'},
 {'comments': '1860098',
  'rank': '8',
  'rating_num': '9.4',
  'rating_star': 'rating45',
  'title': '星际穿越'},
 {'comments': '1113679',
  'rank': '9',
  'rating_num': '9.6',
  'rating_star': 'rating5',
  'title': '

In [10]:
 # 执行所有的HTML的页面解析
all_datas=[]
for html in htmls:
    all_datas.extend(parse_singe_html(html))

In [11]:
all_datas

[{'rank': '1',
  'title': '肖申克的救赎',
  'rating_star': 'rating5',
  'rating_num': '9.7',
  'comments': '2919828'},
 {'rank': '2',
  'title': '霸王别姬',
  'rating_star': 'rating5',
  'rating_num': '9.6',
  'comments': '2155515'},
 {'rank': '3',
  'title': '阿甘正传',
  'rating_star': 'rating5',
  'rating_num': '9.5',
  'comments': '2176753'},
 {'rank': '4',
  'title': '泰坦尼克号',
  'rating_star': 'rating5',
  'rating_num': '9.5',
  'comments': '2209162'},
 {'rank': '5',
  'title': '这个杀手不太冷',
  'rating_star': 'rating45',
  'rating_num': '9.4',
  'comments': '2310124'},
 {'rank': '6',
  'title': '千与千寻',
  'rating_star': 'rating45',
  'rating_num': '9.4',
  'comments': '2260626'},
 {'rank': '7',
  'title': '美丽人生',
  'rating_star': 'rating5',
  'rating_num': '9.6',
  'comments': '1335427'},
 {'rank': '8',
  'title': '星际穿越',
  'rating_star': 'rating45',
  'rating_num': '9.4',
  'comments': '1860098'},
 {'rank': '9',
  'title': '辛德勒的名单',
  'rating_star': 'rating5',
  'rating_num': '9.6',
  'comments': '1

In [13]:
#将结果存入Excel
df=pd.DataFrame(all_datas)
df.to_excel('./豆瓣电影爬虫完结.xlsx')
df

,rank,title,rating_star,rating_num,comments
0,1,肖申克的救赎,rating5,9.7,2919828
1,2,霸王别姬,rating5,9.6,2155515
2,3,阿甘正传,rating5,9.5,2176753
3,4,泰坦尼克号,rating5,9.5,2209162
4,5,这个杀手不太冷,rating45,9.4,2310124
...,...,...,...,...,...
245,246,朗读者,rating45,8.6,455349
246,247,燃情岁月,rating45,8.8,288261
247,248,再次出发之纽约遇见你,rating45,8.6,429025
248,249,香水,rating45,8.5,546313
